In [30]:
#Import Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser

In [31]:
#Scrape NASA Mars News Site
#Set the executable path
executable_path = {'executable_path':'../../chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#Set the url to visit
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [32]:
#Set up BeautifulSoup object
html = browser.html
soup = bs(html, 'html.parser')

In [33]:
#Scrape Latest News Article
#Scrape Title
article = soup.find('div', class_='list_text')
news_title = article.find('a').text
#Scrape article text
news_p = article.find('div', class_='article_teaser_body').text

In [34]:
#Have splinter go to site for image scraping
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url2)
#Reset Beautiful Soup
html2 = browser.html
soup2 = bs(html2, 'html.parser')

In [35]:
#Scrape main image
img_style = soup2.find('article', class_='carousel_item')['style']
img_path = img_style[22:(len(img_style)-2)]
featured_image_url = url2 + img_path

In [46]:
#Set spliter to Mars Weather Twitter
#url3 = 'https://twitter.com/marswxreport?lang=en'
#browser.visit(url3)
#Reset Beautiful Soup
#html3 = browser.html
#soup3 = bs(html3, 'lxml')

In [48]:
#Scrape weather information
#soup3.body.find_all('span', class_='css-901oao css-16my406z').text
#This should work but bs can't seem to parse twitter's page